# Note

- 之前的训练数据是分散的，标签没有对齐；且没有用v8x作为base
- 本次模型训练旨在以v8x为base训练更好的Player和Ball识别
- 数据集：basketball-w2xcw-xmbod(修改了name)

# Data Preparation

In [1]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wKugMGCPQkZaOOt2pxZE")
project = rf.workspace("sponge-hjhsw").project("basketball-w2xcw-xmbod")
version = project.version(1)
dataset = version.download("yolov8", location="../../dataset/Ball_Player_Hoop")
                


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.3.155, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to ../../dataset/Ball_Player_Hoop in yolov8:: 100%|██████████| 8134/8134 [00:03<00:00, 2440.86it/s]


# Environment Preparation

In [1]:
import torch
from ultralytics import YOLO
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

print("CUDA:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0))
print("Total Memory:", torch.cuda.get_device_properties(0).total_memory / 1024**2, "MB")


CUDA: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Total Memory: 8187.5 MB


In [2]:
import os
# 定义路径
model_path = "../../models/yolov8x.pt" 
last_weights_path = "../../runs/detection_train/train_1/weights/last.pt"
data_path = '../../dataset/Ball_Player_Hoop/data.yaml'
project_path = "../../runs/detection_train"

project_path = "../../runs/detection_train"
logdir = os.path.join(project_path, "log")
logdir

'../../runs/detection_train\\log'

In [ ]:
%pip install tensorboard

In [2]:
# 打开TensorBoard
%load_ext tensorboard
%tensorboard --logdir ../../runs/detection_train/log

Reusing TensorBoard on port 6006 (pid 29452), started 9 days, 2:43:44 ago. (Use '!kill 29452' to kill it.)

# Train 1

In [4]:
# 检查是否存在上次训练的检查点
if os.path.exists(last_weights_path):
    print(f"发现检查点文件 {last_weights_path}，将从中断处恢复训练")
    model = YOLO(last_weights_path)  # 加载上次训练的权重
else:
    print(f"未发现检查点文件，将从头开始训练")
    model = YOLO(model_path)  # 加载预训练模型

# 开始训练（自动恢复或从头开始）
model.train(
    data=data_path,
    epochs=250,
    imgsz=640,
    batch=8,
    project=project_path,
    name = "train_1",
    device = 0,
    patience = 100,
    resume=False  # 启用自动恢复功能(需要恢复时用)
)

发现检查点文件 ../../runs/detection_train/train_1/weights/last.pt，将从中断处恢复训练
Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../dataset/Ball_Player_Hoop/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../../runs/detection_train/train_1/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_17, n

train: Scanning D:\AI\yolo\dataset\Ball_Player_Hoop\train\labels.cache... 2843 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2843/2843 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.40.3 ms, read: 20.325.5 MB/s, size: 157.5 KB)


val: Scanning D:\AI\yolo\dataset\Ball_Player_Hoop\valid\labels.cache... 812 images, 0 backgrounds, 0 corrupt: 100%|██████████| 812/812 [00:00<?, ?it/s]


Plotting labels to ..\..\runs\detection_train\train_17\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\..\runs\detection_train\train_17
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        812       3116      0.944       0.94      0.962      0.807



250 epochs completed in 14.094 hours.
Optimizer stripped from ..\..\runs\detection_train\train_17\weights\last.pt, 136.7MB
Optimizer stripped from ..\..\runs\detection_train\train_17\weights\best.pt, 136.7MB

Validating ..\..\runs\detection_train\train_17\weights\best.pt...
Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 112 layers, 68,126,457 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:16<00:00,  3.08it/s]


                   all        812       3116      0.945      0.932      0.962      0.812
                  Ball        689        802       0.94       0.91      0.947      0.777
                  Hoop        527        543      0.985      0.971      0.989      0.873
                Player        579       1771       0.91      0.916      0.951      0.785
Speed: 0.2ms preprocess, 15.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to ..\..\runs\detection_train\train_17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001762D678940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

# Val

In [3]:
# 加载训练好的模型
model = YOLO("../../models/v8x_player_ball.pt")

# 配置验证参数
results = model.val(
    data="../../dataset/Ball_Player_Hoop/data.yaml",  # 数据集配置文件路径
    imgsz=640,                                           # 输入图像尺寸
    batch=8,                                             # 每批次的图像数量
    project="../../runs/detection_val"                       # 验证结果保存的路径
)

Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 112 layers, 68,126,457 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access  (ping: 0.30.0 ms, read: 30.326.9 MB/s, size: 227.5 KB)


val: Scanning D:\AI\yolo\dataset\Ball_Player_Hoop\valid\labels.cache... 812 images, 0 backgrounds, 0 corrupt: 100%|██████████| 812/812 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 102/102 [00:30<00:00,  3.38it/s]


                   all        812       3116      0.945      0.932      0.962      0.813
                  Ball        689        802      0.941       0.91      0.947      0.779
                  Hoop        527        543      0.985      0.971      0.989      0.875
                Player        579       1771       0.91      0.916      0.951      0.786
Speed: 0.2ms preprocess, 32.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to ..\..\runs\detection_val\val


# Test


In [6]:
model = YOLO("../../models/v8x_player_ball.pt")
results = model.track("../../sources/game_clip1.mp4", show=True)

for result in results:
    print(result[0].names)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/318) d:\AI\yolo\code\notebook\..\..\sources\game_clip1.mp4: 384x640 2 Balls, 9 Players, 86.3ms
video 1/1 (frame 2/318) d:\AI\yolo\code\notebook\..\..\sources\game_clip1.mp4: 384x640 2 Balls, 9 Players, 41.5ms
video 1/1 (frame 3/318) d:\AI\yolo\code\notebook\..\..\sources\game_clip1.mp4: 384x640 2 Balls, 9 Players, 41.2ms
video 1/1 (frame 4/318) d:\AI\yolo\code\notebook\..\..\sources\game_clip1.mp4: 384x640 10 Players, 41.2ms
video 1/1 (

# Train 2


In [3]:
import os
# 定义路径
model_path = "../../models/yolo11x.pt" 
last_weights_path = "../../runs/detection_train/train_2/weights/last.pt"
data_path = '../../dataset/Ball_Player_Hoop/data.yaml'
project_path = "../../runs/detection_train"

project_path = "../../runs/detection_train"
logdir = os.path.join(project_path, "log")
logdir

'../../runs/detection_train\\log'

In [ ]:
# 检查是否存在上次训练的检查点
if os.path.exists(last_weights_path):
    print(f"发现检查点文件 {last_weights_path}，将从中断处恢复训练")
    model = YOLO(last_weights_path)  # 加载上次训练的权重
else:
    print(f"未发现检查点文件，将从头开始训练")
    model = YOLO(model_path)  # 加载预训练模型

# 开始训练（自动恢复或从头开始）
model.train(
    data=data_path,
    epochs=250,
    imgsz=640,
    batch=8,
    project=project_path,
    name = "train_2",
    device = 0,
    patience = 100,
    resume=False  # 启用自动恢复功能(需要恢复时用)
)